<h1>Capstone Project - The Battle of Neighborhoods</h>

<h2>Fred Vezzani</h2>

<h3>Introduction: Coffee Place in Milan</h3>

In one of the world's Coffee Capitals, the Coffee Place World is very crowded.
A small entrepreneur in Milan has told me to explore his chances

<h3>Data:</h3>

We will need to connect to FourSquare API to get addresses, and lat/long of each Coffee Place. Then put them on a Folium Map

<h3>Methodology:</h3>

In [7]:
# Importing Libs
import requests 
import pandas as pd
import numpy as np
!pip install folium
import folium 

In [8]:
# FourSquare Creds
CLIENT_ID = 'CBQBV2BD34KTDEZQFLNOKPLHNBG15GWXLHKMIXE2HAQC4GG5'
CLIENT_SECRET = 'B4AU53NJZJA5OYA1KGOLSEBHDFEOOR1KHKUNIA03MHYHF4J4'
VERSION = '20200801'
LIMIT = 40

In [16]:
# Explore Milan
request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Milan",
    "radius": 1000,
    "limit": 50}
data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [17]:
# Analyzing Response
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [18]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Milan', 'Milan')

In [19]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 45.47109412052524, 'lng': 9.200040988476063},
  'sw': {'lat': 45.457643655196364, 'lng': 9.17989738150114}},
 143)

In [20]:
d["geocode"]

{'what': '',
 'where': 'milan',
 'center': {'lat': 45.46427, 'lng': 9.18951},
 'displayString': 'Milan, Lombardy, Italy',
 'cc': 'IT',
 'geometry': {'bounds': {'ne': {'lat': 45.53644193307576,
    'lng': 9.278070963403922},
   'sw': {'lat': 45.38920202383928, 'lng': 9.039837068386658}}},
 'slug': 'milano-italy',
 'longId': '72057594041101371'}

In [21]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [22]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [23]:
# Creating Items of Coffee Shop and printing one
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '57d52948498ebceb900cf11c',
  'name': 'Pasticceria Marchesi',
  'location': {'address': 'Galleria Vittorio Emanuele II',
   'lat': 45.46561245831536,
   'lng': 9.190091221105353,
   'labeledLatLngs': [{'label': 'display',
     'lat': 45.46561245831536,
     'lng': 9.190091221105353}],
   'postalCode': '20121',
   'cc': 'IT',
   'neighborhood': 'Duomo',
   'city': 'Milano',
   'state': 'Lombardia',
   'country': 'Italia',
   'formattedAddress': ['Galleria Vittorio Emanuele II',
    '20121 Milano Lombardia',
    'Italia']},
  'categories': [{'id': '5744ccdfe4b0c0459246b4e2',
    'name': 'Pastry Shop',
    'pluralName': 'Pastry Shops',
    'shortName': 'Pastry',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/dessert_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 're

In [24]:
# Time to create the Dataframe
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

{'address': 'Galleria Vittorio Emanuele II', 'lat': 45.46561245831536, 'lng': 9.190091221105353, 'labeledLatLngs': [{'label': 'display', 'lat': 45.46561245831536, 'lng': 9.190091221105353}], 'postalCode': '20121', 'cc': 'IT', 'neighborhood': 'Duomo', 'city': 'Milano', 'state': 'Lombardia', 'country': 'Italia', 'formattedAddress': ['Galleria Vittorio Emanuele II', '20121 Milano Lombardia', 'Italia']}
{'address': 'Piazza Cordusio 3', 'crossStreet': 'Via Orefici', 'lat': 45.46492026479228, 'lng': 9.186152815818787, 'labeledLatLngs': [{'label': 'display', 'lat': 45.46492026479228, 'lng': 9.186152815818787}], 'postalCode': '20123', 'cc': 'IT', 'city': 'Milano', 'state': 'Lombardia', 'country': 'Italia', 'formattedAddress': ['Piazza Cordusio 3 (Via Orefici)', '20123 Milano Lombardia', 'Italia']}
{'address': 'Piazza San Fedele, 2', 'lat': 45.46627391020354, 'lng': 9.190975102861223, 'labeledLatLngs': [{'label': 'display', 'lat': 45.46627391020354, 'lng': 9.190975102861223}], 'postalCode': '20

,uid,name,shortname,address,postalcode,lat,lng
0,57d52948498ebceb900cf11c,Pasticceria Marchesi,Pastry,,20121,45.465612,9.190091
1,58b5d3364e31c52f888a0bed,Starbucks Reserve Roastery,Coffee Shop,,20123,45.464920,9.186153
2,59c2232bd48ec17dd4178369,Lavazza Coffee Design,Coffee Shop,,20122,45.466274,9.190975
3,5485c0d7498e225e0587b79b,Panini Durini,Sandwiches,,20121,45.465238,9.188590
4,4be181c240d676b02de504ee,Princi,Bakery,,20123,45.463029,9.187968


In [25]:
# Setting Milan Location
Milan = d["geocode"]["center"]
Milan

{'lat': 45.46427, 'lng': 9.18951}

In [28]:
# Showing Milan Map 

import folium.plugins as plugins
map_Milan = folium.Map(location=[45.46427, 9.18951], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Milan)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_Milan.add_child(plugins.HeatMap(hm_data))

map_Milan

<h3>Conclusion:</h3>

The HeatMap can tell us exactly where to open a new Caffee. It should be close to the hottest points on the Map, since they are the most trendy but at the same time, not inside the Heat or there will be too much competition